In [40]:
#importing libraries
import os
import pandas as pd

In [41]:
staging_folder = r'C:\Users\JenishaE\Documents\finalDE\finalProject-Data\staging'
prep_folder = r'C:\Users\JenishaE\Documents\finalDE\finalProject-Data\prep'

In [42]:
if not os.path.exists(prep_folder):
    os.makedirs(prep_folder)

In [43]:
#path of the file
file_path=os.path.join(staging_folder,'loaded_employee_data.csv')

In [44]:
##convert file to dataframe
df_employee_data=pd.read_csv(file_path)

In [45]:
#cols
df_employee_data.columns

Index(['emp_id', 'emp_name', 'department', 'designation', 'DOJ', 'age',
       'experience', 'salary', 'no_of_promotions', 'education'],
      dtype='object')

In [46]:
#info
df_employee_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   emp_id            5000 non-null   object 
 1   emp_name          5000 non-null   object 
 2   department        5000 non-null   object 
 3   designation       5000 non-null   object 
 4   DOJ               5000 non-null   object 
 5   age               5000 non-null   int64  
 6   experience        4848 non-null   float64
 7   salary            4840 non-null   float64
 8   no_of_promotions  4851 non-null   float64
 9   education         5000 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 390.8+ KB


In [47]:
#converting the data types
df_employee_data['emp_id'] = df_employee_data['emp_id'].astype('string')
df_employee_data['emp_name'] = df_employee_data['emp_name'].astype('string')
df_employee_data['department'] = df_employee_data['department'].astype('string')
df_employee_data['designation'] = df_employee_data['designation'].astype('string')
df_employee_data['DOJ'] = pd.to_datetime(df_employee_data['DOJ'], format='%Y-%m-%d').dt.date  # Date only
df_employee_data['age'] = df_employee_data['age'].astype('int')
df_employee_data['experience'] = df_employee_data['experience'].astype('Int64')  # Nullable integer
df_employee_data['salary'] = df_employee_data['salary'].astype('Int64')  # Nullable integer
df_employee_data['no_of_promotions'] = df_employee_data['no_of_promotions'].astype('Int64')  # Nullable integer
df_employee_data['education'] = df_employee_data['education'].astype('string')

In [48]:
df_employee_data.head()

,emp_id,emp_name,department,designation,DOJ,age,experience,salary,no_of_promotions,education
0,E001,Sharon Burns,Finance,Senior Developer,2006-11-22,45,11,95402,1,Bachelors
1,E002,Hannah Sanchez,Finance,Senior Developer,2010-02-25,41,11,89438,3,Masters
2,E003,Larry Waters,Marketing,Senior Developer,2019-10-19,44,11,91396,2,Masters
3,E004,Angela Delacruz,Sales,Junior Developer,2014-08-27,24,2,23081,0,Bachelors
4,E005,John Cook,HR,Junior Developer,2009-11-20,24,2,22370,0,Masters


In [49]:
#no.of.rows and cols
df_employee_data.shape

(5000, 10)

In [50]:
df_employee_data.dtypes

emp_id              string[python]
emp_name            string[python]
department          string[python]
designation         string[python]
DOJ                         object
age                          int64
experience                   Int64
salary                       Int64
no_of_promotions             Int64
education           string[python]
dtype: object

In [51]:
#no.of.null values in each col
df_employee_data.isnull().sum()


emp_id                0
emp_name              0
department            0
designation           0
DOJ                   0
age                   0
experience          152
salary              160
no_of_promotions    149
education             0
dtype: int64

In [52]:
#removing the duplicate rows based on emp_id
df_employee_data = df_employee_data.drop_duplicates(subset='emp_id', keep='first')

In [53]:
print(df_employee_data['age'].min())
print(df_employee_data['age'].max())

22
50


In [54]:
#filling the experience based on age
df_employee_data['experience'] = df_employee_data['experience'].fillna(df_employee_data['age'] - 21)#considering employee with 22 years have 1 yr of experience

In [55]:
df_employee_data['experience'] = df_employee_data['experience'].apply(lambda x: max(x, 0))#converting to 0 if experience is a negative value

In [56]:
mean_salary_by_experience = df_employee_data.groupby('experience')['salary'].mean()
print(mean_salary_by_experience)

experience
0      27936.512048
1      27080.236025
2      27168.438889
3      42376.238754
4      42123.731788
5      42096.797048
6      64063.874251
7          41064.25
8           44638.0
9           61894.4
10       65559.4825
11     99991.881564
12     64551.111111
13          59261.0
14     60021.166667
15         67711.75
16          71250.2
17     62458.333333
18     63322.666667
19    101237.833333
20     99581.415476
21    101532.333333
22          95753.4
23         102132.8
24        105301.75
25    102390.285714
26         102480.5
27          94004.0
28          92160.2
29          97107.2
Name: salary, dtype: Float64


In [57]:
def fill_salary(row):
    if pd.isnull(row['salary']):
        # Look up the mean salary for the employee's experience
        return mean_salary_by_experience.get(row['experience'], df_employee_data['salary'].mean())
    return row['salary'] 

In [58]:
#filling the null values in the salary col with mean value based on experience
df_employee_data['salary'] = df_employee_data.apply(fill_salary, axis=1)

In [59]:
print(df_employee_data['experience'].min())
print(df_employee_data['experience'].max())

0
29


In [60]:
#filling the null values in promotion
for i in range(len(df_employee_data)):
    experience = df_employee_data.loc[i, 'experience']
    
    if pd.isnull(df_employee_data.loc[i, 'no_of_promotions']):
        # Set promotion based on experience logic
        if experience < 5:
            df_employee_data.loc[i, 'no_of_promotions'] = 0
        elif 5 <= experience < 10:
            df_employee_data.loc[i, 'no_of_promotions'] = 1
        elif 10 <= experience < 15:
            df_employee_data.loc[i, 'no_of_promotions'] = 2
        elif 15 <= experience < 20:
            df_employee_data.loc[i, 'no_of_promotions'] = 4
        elif 20 <= experience < 25:
            df_employee_data.loc[i, 'no_of_promotions'] = 5
        else:  # experience >= 25
            df_employee_data.loc[i, 'no_of_promotions'] = 6

In [61]:
df_employee_data.isnull().sum()

emp_id              0
emp_name            0
department          0
designation         0
DOJ                 0
age                 0
experience          0
salary              0
no_of_promotions    0
education           0
dtype: int64

In [62]:
df_employee_data.shape

(5000, 10)

In [63]:
#move the cleaned file into prep folder
target_path=os.path.join(prep_folder,'emp_data_prep.csv')
df_employee_data.to_csv(target_path, index=False)